# Simple Fully Connected Autoencoder

In [6]:
from river import compose, preprocessing, metrics, datasets

from deep_river.anomaly import Autoencoder, AutoencoderInitialized
from torch import nn, manual_seed

In [7]:
_ = manual_seed(42)
dataset = datasets.CreditCard().take(5000)
metric = metrics.RollingROCAUC(window_size=5000)


class MyAutoEncoder(nn.Module):
    def __init__(self, n_features, latent_dim=3):
        super(MyAutoEncoder, self).__init__()
        self.linear1 = nn.Linear(n_features, latent_dim)
        self.nonlin = nn.LeakyReLU()
        self.linear2 = nn.Linear(latent_dim, n_features)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X, **kwargs):
        X = self.linear1(X)
        X = self.nonlin(X)
        X = self.linear2(X)
        return self.sigmoid(X)


model_pipeline = compose.Pipeline(
    preprocessing.MinMaxScaler(), AutoencoderInitialized(module=MyAutoEncoder(30), lr=0.005)
)
model_pipeline

Pipeline (
  MinMaxScaler (),
  AutoencoderInitialized (
    module=MyAutoEncoder(
    (linear1): Linear(in_features=30, out_features=3, bias=True)
    (nonlin): LeakyReLU(negative_slope=0.01)
    (linear2): Linear(in_features=3, out_features=30, bias=True)
    (sigmoid): Sigmoid()
  )
    loss_fn="mse"
    optimizer_fn="sgd"
    lr=0.005
    is_feature_incremental=False
    device="cpu"
    seed=42
  )
)

In [8]:
for x, y in dataset:
    score = model_pipeline.score_one(x)
    metric.update(y_true=y, y_pred=score)
    model_pipeline.learn_one(x=x)
print(f"ROCAUC: {metric.get():.4f}")

ROCAUC: 0.8901
